In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession

spark = (SparkSession.builder.
         appName("working with tabular data").
         getOrCreate())

In [3]:
import os
import pyspark.sql.functions as F

In [4]:
DIRECTORY = r"C:\Users\oluwa\Documents\broadcast_logs"

In [7]:
logs = spark.read.csv(
    os.path.join(DIRECTORY, "BroadcastLogs_2018_Q3_M8.csv"),
    sep = "|",
    header = True,
    inferSchema = True,
    timestampFormat = "yyyy-MM-dd",
)